In [1]:
from random import choices,randint,randrange,random
from typing import List,Callable
from collections import namedtuple
from functools import partial

In [2]:
Genome = List[int]
Population = List[Genome]
PopulateFunc = Callable[[], Population]
FitnessFunc = Callable[[Genome], int]
SelectionFunc = Callable[[Population, FitnessFunc], tuple[Genome,Genome]]
CrossoverFunc = Callable[[Genome,Genome],tuple[Genome,Genome]]
MutationFunc = Callable[[Genome],Genome]
Thing = namedtuple('Thing',['name','value','weight'])

things = [
    Thing('Laptop',500,2200),
    Thing('Headphones',150,160),
    Thing('Mug',60,350),
    Thing('Notepad',40,333),
    Thing('Bottle',30,192),
    Thing('Hammer',300,2000),
    Thing('Phone',800,400)
]

In [3]:
def generate_genome(length: int)-> Genome:
    return choices([0,1], k=length)

def generate_population(size:int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]

In [4]:
def fitness(genome: Genome, things: list[Thing], weight_limit: int) -> int:
    if len(genome) != len(things):
        raise ValueError("genome and things must be of same length")
    
    weight = 0
    value = 0

    for i, thing in enumerate(things):
        if genome[i] == 1:
            weight+= thing.weight
            value+= thing.value

            if weight> weight_limit:
                return 0
            
    return value

In [5]:
def selection_pair(population: Population, fitness_function: FitnessFunc) -> Population:
    return choices(
        population=population,
        weights=[fitness_function(genome) for genome in population],
        k = 2
    )

In [6]:
def single_point_crossover(a: Genome, b:Genome) -> tuple[Genome,Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of the same length")
    
    length = len(a)
    if length < 2:
        return a,b
    
    p = randint(1, length-1)
    return a[0:p] + b[p:], b[0:p] + a[p:]

In [7]:
def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index= randrange(len(genome))
        if random() <= probability:
            mutated_value = abs(genome[index] - 1)
        else:
            mutated_value = genome[index]
        genome[index] = mutated_value
    return genome

In [8]:
def run_evolution(
        populate_func:PopulateFunc,
        fitness_func:FitnessFunc,
        fitness_limit:int,
        selection_func:SelectionFunc = selection_pair,
        crossover_func:CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        generation_limit: int = 100
) -> tuple[Population, int]:
        
        population = populate_func()
        
        for i in range(generation_limit):
                population = sorted(
                        population,
                        key= lambda genome: fitness_func(genome),
                        reverse= True
                )

                if fitness_func(population[0]) >= fitness_limit:
                        break

                next_generation = population[0:2]

                for j in range(int(len(population)/2)-1):
                        parents = selection_func(population, fitness_func)
                        offspring_a, offspring_b = crossover_func(parents[0],parents[1])
                        offspring_a = mutation_func(offspring_a)
                        offspring_b = mutation_func(offspring_b)
                        next_generation += [offspring_a, offspring_b]

                population = next_generation
        
        population = sorted(
                population,
                key= lambda genome: fitness_func(genome),
                reverse= True
        )

        return population,i

In [15]:
population, generations = run_evolution(
    populate_func= partial(generate_population, size=10, genome_length=len(things)),
    fitness_func= partial(fitness, things=things, weight_limit=3500),
    fitness_limit=1550,
    generation_limit=100
)

In [10]:
def genome_to_things(genome:Genome, things: list[Thing]) -> list[Thing]:
    result = []
    for i, thing in enumerate(things):
        if genome[i] == 1:
            result += [thing.name]

    return result

In [18]:
print(f"number of generations: {generations}")
print(f"best solutions: {genome_to_things(population[0], things)}")

number of generations: 10
best solutions: ['Laptop', 'Headphones', 'Mug', 'Notepad', 'Phone']
